In [1]:
import pandas as pd
import numpy as np
from fyers_apiv3 import fyersModel
import tensorflow as tf
from indicators import alligator, vroc, sma, epoch_to_time, time_to_epoch
import pandas as pd
import numpy as np
import time, datetime
client_id = "ZHQ4IJL7TI-100"
model = tf.keras.models.load_model("07OCT.keras")
with open("access_token", "r") as f:
    access_token = f.read()

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

In [2]:
date_today = datetime.datetime.now().strftime("%Y-%m-%d")
date_100_p = (datetime.datetime.now() - datetime.timedelta(days=100)).strftime("%Y-%m-%d")
date_today, date_100_p

('2024-11-19', '2024-08-11')

In [3]:
response = fyers.history(data={"symbol": "NSE:NIFTY24OCT24200CE",
                                "resolution": "1",
                                 "date_format": "1",
                                "range_from": "2024-10-17",
                                "range_to": "2024-10-25",
                                "cont_flag": "1"
                                })

len(response['candles']), response

(2621,
 {'candles': [[1729136760, 880.05, 880.05, 880, 880, 150],
   [1729136820, 880, 880, 880, 880, 50],
   [1729136880, 880, 880, 880, 880, 0],
   [1729136940, 880, 880, 880, 880, 0],
   [1729137000, 880, 880, 880, 880, 0],
   [1729137060, 880, 880, 880, 880, 0],
   [1729137120, 880, 880, 880, 880, 0],
   [1729137180, 880, 880, 880, 880, 0],
   [1729137240, 880, 880, 880, 880, 0],
   [1729137300, 880, 880, 880, 880, 0],
   [1729137360, 880, 880, 880, 880, 0],
   [1729137420, 880, 880, 880, 880, 0],
   [1729137480, 836, 836, 836, 836, 25],
   [1729137540, 836, 836, 836, 836, 0],
   [1729137600, 836, 836, 836, 836, 0],
   [1729137660, 836, 836, 836, 836, 0],
   [1729137720, 836, 836, 836, 836, 0],
   [1729137780, 836, 836, 836, 836, 0],
   [1729137840, 836, 836, 836, 836, 0],
   [1729137900, 836, 836, 808, 808, 50],
   [1729137960, 808, 808, 808, 808, 0],
   [1729138020, 808, 808, 808, 808, 0],
   [1729138080, 808, 808, 808, 808, 0],
   [1729138140, 808, 808, 808, 808, 0],
   [1729138

In [4]:
for i in response['candles']:
    cnt = 0
    for j in response['candles']:
        if i[0] == j[0]:
            cnt += 1
    if cnt > 1:
        print("Duplicates found")
        raise ValueError("Duplicates found in the candle data")

In [5]:
temp = []
for c in response['candles']:
    temp.append(sum(c[2:4])/2)

alli = alligator(pd.Series(temp))
alli = alli.to_numpy()
alli

array([[         nan,          nan,          nan],
       [         nan,          nan,   0.        ],
       [         nan,          nan,   0.        ],
       ...,
       [121.74738984, 134.92160891, 161.15676096],
       [122.39374447, 136.86578279, 161.52540876],
       [122.96153335, 139.13255994, 161.35532701]])

In [6]:
dates = []
X = []
k = 0

for i in range(len(response['candles'])-1):
# for i in range(10):
    candle = response["candles"][i]
    d = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(candle[0]))
    if d not in dates:
        dates.append(d)
        temp = []
        temp.append(d)
        temp.append(candle[1])
        temp.append(candle[2])
        temp.append(candle[3])
        temp.append(candle[4])
        temp.append(round(alli[i][0], 2))
        temp.append(round(alli[i][1], 2))
        temp.append(round(alli[i][2], 2))
        temp.append(round(sma(np.array(response['candles'])[:i+1, 4], 5)[-1], 2))
        temp.append(round(sma(np.array(response['candles'])[:i+1, 4], 15)[-1], 2))
        temp.append(round(vroc(np.array(response['candles'])[:i+1, 5], 14), 2))
        temp.append(candle[5])
        X.append(temp)
        print("id:", i, "\tdata:", temp)
    else:
        k += 1
        print("skipping", k, "\t", d)

id: 0 	data: ['2024-10-17 09:16:00', 880.05, 880.05, 880, 880, nan, nan, nan, nan, nan, nan, 150]
id: 1 	data: ['2024-10-17 09:17:00', 880, 880, 880, 880, nan, nan, 0.0, nan, nan, nan, 50]
id: 2 	data: ['2024-10-17 09:18:00', 880, 880, 880, 880, nan, nan, 0.0, nan, nan, nan, 0]
id: 3 	data: ['2024-10-17 09:19:00', 880, 880, 880, 880, nan, nan, 0.0, nan, nan, nan, 0]
id: 4 	data: ['2024-10-17 09:20:00', 880, 880, 880, 880, nan, nan, 0.0, 880.0, nan, nan, 0]
id: 5 	data: ['2024-10-17 09:21:00', 880, 880, 880, 880, nan, nan, 880.0, 880.0, nan, nan, 0]
id: 6 	data: ['2024-10-17 09:22:00', 880, 880, 880, 880, nan, nan, 880.0, 880.0, nan, nan, 0]
id: 7 	data: ['2024-10-17 09:23:00', 880, 880, 880, 880, nan, nan, 880.0, 880.0, nan, nan, 0]
id: 8 	data: ['2024-10-17 09:24:00', 880, 880, 880, 880, nan, nan, 880.0, 880.0, nan, nan, 0]
id: 9 	data: ['2024-10-17 09:25:00', 880, 880, 880, 880, nan, nan, 880.0, 880.0, nan, nan, 0]
id: 10 	data: ['2024-10-17 09:26:00', 880, 880, 880, 880, nan, nan, 8

/Users/anodic_passion/PycharmProjects/108decillion/indicators/__init__.py:34: RuntimeWarning: invalid value encountered in scalar divide
  vroc = ((V_t - V_tn) / V_tn) * 100
/Users/anodic_passion/PycharmProjects/108decillion/indicators/__init__.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  vroc = ((V_t - V_tn) / V_tn) * 100


id: 175 	data: ['2024-10-17 12:11:00', 718.4, 720.95, 718.4, 720.95, 705.67, 710.89, 712.55, 714.83, 713.82, -75.0, 25]
id: 176 	data: ['2024-10-17 12:12:00', 720.95, 720.95, 720.95, 720.95, 705.91, 711.52, 713.98, 716.7, 714.16, nan, 0]
id: 177 	data: ['2024-10-17 12:13:00', 720.95, 720.95, 720.95, 720.95, 706.68, 712.06, 715.37, 718.57, 714.5, nan, 0]
id: 178 	data: ['2024-10-17 12:14:00', 720.95, 720.95, 720.95, 720.95, 707.39, 712.01, 716.49, 720.44, 714.83, nan, 0]
id: 179 	data: ['2024-10-17 12:15:00', 720.95, 720.95, 720.95, 720.95, 708.04, 711.95, 717.38, 720.95, 715.17, -100.0, 0]
id: 180 	data: ['2024-10-17 12:16:00', 720.95, 720.95, 720.95, 720.95, 708.65, 711.91, 718.09, 720.95, 715.79, nan, 0]
id: 181 	data: ['2024-10-17 12:17:00', 720.95, 720.95, 720.95, 720.95, 709.21, 711.87, 718.67, 720.95, 716.42, nan, 0]
id: 182 	data: ['2024-10-17 12:18:00', 720.95, 720.95, 715.2, 715.2, 709.72, 711.84, 719.12, 719.8, 716.66, inf, 25]
id: 183 	data: ['2024-10-17 12:19:00', 715.2, 71

In [7]:
data = np.array(X)[1496:, :-1][:]
data

array([['2024-10-23 09:15:00', '441', '441', ..., '425.62', '426.49',
        '5.75'],
       ['2024-10-23 09:16:00', '419.05', '422.55', ..., '424.45',
        '426.51', '115.0'],
       ['2024-10-23 09:17:00', '413.3', '444.35', ..., '427.03', '427.3',
        '114.29'],
       ...,
       ['2024-10-25 15:26:00', '165.65', '165.75', ..., '163.55',
        '158.12', '-73.95'],
       ['2024-10-25 15:27:00', '162.05', '164.7', ..., '163.33',
        '158.78', '-39.02'],
       ['2024-10-25 15:28:00', '162.1', '162.4', ..., '162.57', '159.74',
        '-30.19']], dtype='<U32')

In [8]:
X, y = [], []
last_candles_count = 8
next_candles_count = 3

for idx, row in enumerate(data[last_candles_count - 1:-(next_candles_count + 1)]):
    idx = idx + last_candles_count + 1
    first_open = data[idx - last_candles_count - 1][0]
    first_jaw = data[idx - last_candles_count - 1][4]
    first_lip = data[idx - last_candles_count - 1][6]
    first_teeth = data[idx - last_candles_count - 1][5]
    first_ma5 = data[idx - last_candles_count - 1][7]
    first_ma15 = data[idx - last_candles_count - 1][8]
    first_volumn_change = data[idx - last_candles_count - 1][9]
    last_candles = []
    for candle in data[idx - last_candles_count: idx]:
        temp = []
        temp.append(round(candle[0] - first_open, 2))
        temp.append(round(candle[1] - first_open, 2))
        temp.append(round(candle[2] - first_open, 2))
        temp.append(round(candle[3] - first_open, 2))
        temp.append(round(candle[4] - first_jaw, 2))
        temp.append(round(candle[5] - first_teeth, 2))
        temp.append(round(candle[6] - first_lip, 2))
        temp.append(round(candle[7] - first_ma5, 2))
        temp.append(round(candle[8] - first_ma15, 2))
        temp.append(round(candle[9] - first_volumn_change, 2))

        last_candles.append(temp)
    X.append(last_candles)
    next_candles = []
    for candle in data[idx: idx + next_candles_count]:
        temp = []
        temp.append(round(candle[0] - first_open, 2))

        next_candles.append(temp[0])
    y.append(next_candles)

print("data:\n", data[:8])
print("-1 data:\n", data[-1])
print("\nX:", X[0])
print("\ny:", y[0])
print("X.shape:", np.array(X).shape, "y.shape:", np.array(y).shape)

TypeError: unsupported operand type(s) for -: 'numpy.str_' and 'numpy.str_'

In [ ]:

X = tf.constant(X, dtype=tf.float32)
y = tf.constant(y, dtype=tf.float32)
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y.numpy(), test_size=0.05, random_state=42)

model = Sequential()
model.add(Input(shape=(last_candles_count, 10)))
model.add(LSTM(200, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(150, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(next_candles_count))
# model.add(Reshape((3, 11)))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

model.save("08OCT.keras")